# Day 11: Dumbo Octopus
There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full.
The energy level of each octopus is a value between 0 and 9.
During a single step, the following occurs:

First, the energy level of each octopus increases by 1.
Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.
Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy.
How many total flashes are there after 100 steps?

In [ ]:
let inline stepGrid (grid:seq<seq<int*int>>) =
    let inline bump (b,carry) = match b + 1 with 10 -> (0, carry + 1) | a -> (a,carry)
    seq {
        for y = 0 to 9 do
            yield grid |> Seq.item y |> Seq.map bump
    }
    // |> Array2D.mapi bump 

let rec runSteps max acc grid = 
    if acc = max then grid
    else grid |> stepGrid |> runSteps max (acc+1)

System.IO.File.ReadAllLines("../Data/Day11.txt")
|> Array.map (fun line -> Seq.map (fun c -> (int c - int '0', 0)) line)
|> runSteps 10 0

index,value
0,"[ ( 4, 1 ), ( 1, 1 ), ( 1, 1 ), ( 2, 1 ), ( 2, 1 ), ( 5, 1 ), ( 6, 1 ), ( 3, 1 ), ( 7, 1 ), ( 2, 1 ) ]"
1,"[ ( 3, 1 ), ( 1, 1 ), ( 4, 1 ), ( 3, 1 ), ( 2, 1 ), ( 5, 1 ), ( 3, 1 ), ( 7, 1 ), ( 1, 1 ), ( 2, 1 ) ]"
2,"[ ( 4, 1 ), ( 5, 1 ), ( 1, 1 ), ( 6, 1 ), ( 8, 1 ), ( 4, 1 ), ( 8, 1 ), ( 6, 1 ), ( 3, 1 ), ( 1, 1 ) ]"
3,"[ ( 3, 1 ), ( 7, 1 ), ( 8, 1 ), ( 3, 1 ), ( 4, 1 ), ( 7, 1 ), ( 7, 1 ), ( 1, 1 ), ( 3, 1 ), ( 7, 1 ) ]"
4,"[ ( 3, 1 ), ( 7, 1 ), ( 4, 1 ), ( 6, 1 ), ( 7, 1 ), ( 2, 1 ), ( 3, 1 ), ( 5, 1 ), ( 8, 1 ), ( 2, 1 ) ]"
5,"[ ( 5, 1 ), ( 8, 1 ), ( 6, 1 ), ( 1, 1 ), ( 3, 1 ), ( 5, 1 ), ( 8, 1 ), ( 8, 1 ), ( 8, 1 ), ( 4, 1 ) ]"
6,"[ ( 4, 1 ), ( 8, 1 ), ( 4, 1 ), ( 3, 1 ), ( 3, 1 ), ( 5, 1 ), ( 1, 1 ), ( 7, 1 ), ( 7, 1 ), ( 4, 1 ) ]"
7,"[ ( 2, 1 ), ( 3, 1 ), ( 1, 1 ), ( 6, 1 ), ( 4, 1 ), ( 4, 1 ), ( 7, 1 ), ( 6, 1 ), ( 2, 1 ), ( 1, 1 ) ]"
8,"[ ( 6, 1 ), ( 6, 1 ), ( 4, 1 ), ( 3, 1 ), ( 8, 1 ), ( 1, 1 ), ( 7, 1 ), ( 7, 1 ), ( 4, 1 ), ( 5, 1 ) ]"
9,"[ ( 6, 1 ), ( 3, 1 ), ( 6, 1 ), ( 6, 1 ), ( 8, 1 ), ( 1, 1 ), ( 5, 1 ), ( 8, 1 ), ( 6, 1 ), ( 8, 1 ) ]"
